# Spillover Removal â€“ Marker CSV (normalisierte Pipeline wie V5)

Diese Version repliziert das Verhalten des klassischen Mosaic/Spillover-UI: 
- Komplett-Stack wird pro Kanal auf 0â€“1 normalisiert.
- Spillover-Koeffizienten werden via Mutual Information geschÃ¤tzt und auf den normalisierten Stack angewandt.
- Danach wird jeder Kanal wieder in den ursprÃ¼nglichen Wertebereich zurÃ¼cktransformiert und im Original-Datentyp gespeichert.

Damit sollte die Spillover-Entfernung den bekannten Effekt zeigen.


## 1. Sample Configuration (Dynamisch wie Script 1)

In [ ]:
# === ðŸŽ¯ SAMPLE CONFIGURATION (DYNAMISCH wie Script 1) ===
from pathlib import Path

# === 1. SAMPLE-AUSWAHL ===
current_sample = "sample_004"  # â† HIER SAMPLE Ã„NDERN (muss mit Script 1 Ã¼bereinstimmen)

print(f"ðŸŽ¯ SAMPLE: {current_sample}")

# === 2. DYNAMISCHE PFAD-STRUKTUR (wie Script 1) ===
# Pfade werden automatisch aus current_sample abgeleitet
WORKSPACE_ROOT = Path(r"C:\Users\researcher\data\Epoxy_CyNif\Epoxy_CyNif")
BASE_EXPORT_ROOT = WORKSPACE_ROOT / "data" / "export"
BASE_EXPORT = BASE_EXPORT_ROOT / current_sample

# Legacy-KompatibilitÃ¤t
WORKSPACE_EXPORT_ROOT = BASE_EXPORT_ROOT  
SAMPLE_ID = current_sample

print(f"ðŸ“‚ WORKSPACE_ROOT: {WORKSPACE_ROOT}")
print(f"ðŸ“‚ BASE_EXPORT_ROOT: {BASE_EXPORT_ROOT}")  
print(f"ðŸ“‚ BASE_EXPORT (Sample): {BASE_EXPORT}")

# === 3. VALIDIERUNG ===
if not BASE_EXPORT.exists():
    raise RuntimeError(f"âŒ FEHLER: Sample-Verzeichnis existiert nicht: {BASE_EXPORT}")
else:
    print(f"âœ… Sample-Verzeichnis gefunden")

# === 4. GLOBALE VARIABLEN SETZEN ===
globals()["WORKSPACE_ROOT"] = WORKSPACE_ROOT
globals()["WORKSPACE_EXPORT_ROOT"] = WORKSPACE_EXPORT_ROOT
globals()["BASE_EXPORT_ROOT"] = BASE_EXPORT_ROOT
globals()["BASE_EXPORT"] = BASE_EXPORT
globals()["current_sample"] = current_sample
globals()["SAMPLE_ID"] = SAMPLE_ID

# === INPUT-FILE FINDER (STRENG: Nur aus multicycle_mosaics/decon2D_fused/) ===
MULTICYCLE_DIR = BASE_EXPORT / "multicycle_mosaics"
DECON_DIR = MULTICYCLE_DIR / "decon2D"
FUSED_DIR = DECON_DIR / "decon2D_fused"

if not FUSED_DIR.exists():
    raise RuntimeError(
        f"âŒ FEHLER: FUSED_DIR nicht gefunden: {FUSED_DIR}\n"
        f"   Erwartete Struktur: {BASE_EXPORT}/multicycle_mosaics/decon2D/decon2D_fused/"
    )

# Suche EXPLICIT in FUSED_DIR (NICHT rekursiv!)
input_candidates = sorted(FUSED_DIR.glob("fused_decon*.tif"))  # Auch .tif ohne .ome akzeptieren
if not input_candidates:
    raise FileNotFoundError(
        f"âŒ FEHLER: Kein fused_decon*.tif in {FUSED_DIR} gefunden.\n"
        f"   Script 2 (Spillover) erwartet Input aus Part 1 (Stitching/Decon/EDF).\n"
        f"   Bitte Part 1 ausfÃ¼hren BEVOR Part 2 gestartet wird."
    )
if len(input_candidates) > 1:
    raise RuntimeError(
        f"âŒ FEHLER: Mehrere fused_decon*.ome.tif gefunden in {FUSED_DIR}:\n"
        + '\n'.join(f"   - {p.name}" for p in input_candidates)
        + "\n   Bitte unerwÃ¼nschte Dateien entfernen oder umbenennen."
    )
INPUT_PATH = input_candidates[0]
print(f"âœ… Input-TIF gefunden: {INPUT_PATH.relative_to(BASE_EXPORT)}")

sample_token = ''.join(ch for ch in SAMPLE_ID if ch.isdigit()) or SAMPLE_ID
marker_candidates = [
    csv_path for csv_path in BASE_EXPORT.glob('markers_*.csv')
    if sample_token.lower() in csv_path.stem.lower()
]
if not marker_candidates:
    raise FileNotFoundError(
        f'Erwartete Marker-CSV markers_{sample_token}.csv nicht in {BASE_EXPORT} gefunden.'
    )
if len(marker_candidates) > 1:
    raise RuntimeError(
        'Mehrere Marker-CSV-Dateien gefunden. Bitte unerwÃ¼nschte Dateien entfernen: '
        + ', '.join(str(p.name) for p in marker_candidates)
    )
MARKER_CSV = marker_candidates[0]

ROI_CFG = dict(x=1696, y=4528, size=384)  # hart definierte ROI wie in V7
ROI_PAD = 32  # zusÃ¤tzlicher Kontext fÃ¼r Koeffizienten-SchÃ¤tzung
PERCENTILE_Q = 99.5  # Begrenzung Ã¼ber hohe Percentile (dynamischer Cap)
SAFETY_FACTOR = 1.2  # Sicherheitsfaktor fÃ¼r den Percentile-Cap
MAX_COEFF = 3.0  # hartes globales Maximum als Sicherung

SKIP_CHANNELS = [1, 2, 3]  # 1-basiert
RNG_SEED = 42

print('Konfiguration geladen. Input:', INPUT_PATH)
print('Marker CSV:', MARKER_CSV)

## 2. Imports, Logging, Hilfsfunktionen

In [ ]:
import csv
import json
import logging
import xml.etree.ElementTree as ET
from datetime import datetime
from typing import Any, Dict, List, Tuple, Optional

import imageio.v2 as imageio
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from PIL import Image
from scipy import ndimage
from skimage import filters, morphology
import tifffile as tiff
from tifffile import TiffFile
from ipywidgets import widgets
from tqdm import tqdm

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(message)s',
    datefmt='%H:%M:%S'
 )
logger = logging.getLogger('SpilloverOnly')
logger.info('Logging initialisiert.')

np.random.default_rng(RNG_SEED)

In [ ]:
OME_NS = {'ome': 'http://www.openmicroscopy.org/Schemas/OME/2016-06'}


def extract_channel_names(ome_xml: str, expected: int) -> List[str]:
    names: List[str] = []
    if ome_xml:
        try:
            root = ET.fromstring(ome_xml)
            for ch in root.findall('.//ome:Channel', OME_NS):
                name = ch.get('Name')
                if name:
                    names.append(name)
        except ET.ParseError as exc:
            logger.warning('OME-XML konnte nicht geparst werden: %s', exc)
    if len(names) < expected:
        names.extend([f'C{i+1}' for i in range(len(names), expected)])
    return names[:expected]


def load_ome_to_chw(path: Path) -> Tuple[np.ndarray, np.dtype, List[str]]:
    logger.info('Lade OME-TIFF: %s', path)
    if not path.exists():
        raise FileNotFoundError(path)
    
    with TiffFile(str(path)) as tf:
        ome_xml = tf.ome_metadata
        num_pages = len(tf.pages)
        
        # KRITISCH: PrÃ¼fe ob Multi-Page TIFF (Channels als separate Pages)
        if num_pages > 1:
            logger.info(f'Multi-Page TIFF erkannt: {num_pages} Pages (Channels)')
            # Lade ALLE Pages als separaten Stack
            first_page = tf.pages[0]
            orig_dtype = first_page.dtype
            shape = first_page.shape
            
            # Preallocate Stack: (Channels, Y, X)
            arr = np.empty((num_pages, shape[0], shape[1]), dtype=orig_dtype)
            
            # Lade jede Page in Stack
            logger.info(f'Lade {num_pages} Channels...')
            for i, page in enumerate(tf.pages):
                arr[i] = page.asarray()
                if (i + 1) % 10 == 0:
                    logger.info(f'  {i + 1}/{num_pages} Channels geladen')
            
            logger.info(f'âœ… Alle {num_pages} Channels geladen: Shape={arr.shape}, Dtype={orig_dtype}')
            axes = 'CYX'
        else:
            # Single-Page oder OME mit Achsen
            series = tf.series[0]
            axes = series.axes
            arr = series.asarray()
            orig_dtype = arr.dtype
            
            # Entferne T und Z (nur erster Frame/Plane)
            for ax in ('T', 'Z'):
                if ax in axes:
                    idx = axes.index(ax)
                    arr = np.take(arr, 0, axis=idx)
                    axes = axes.replace(ax, '')
            
            if 'C' not in axes:
                arr = arr[None, ...]
                axes = 'C' + axes
                logger.warning('Keine C-Achse gefunden â€“ nehme erste Dimension als Kanal an.')
            
            c_idx = axes.index('C')
            if c_idx != 0:
                arr = np.moveaxis(arr, c_idx, 0)
            
            # Reduziere auf exakt 3D (C, Y, X)
            while arr.ndim > 3:
                arr = arr[0]
    
    channel_names = extract_channel_names(ome_xml, arr.shape[0])
    return arr.astype(np.float32), orig_dtype, channel_names


def extract_roi_patch(img: np.ndarray, *, x: int, y: int, size: int) -> Tuple[np.ndarray, Tuple[int, int, int, int]]:
    size = int(size)
    half = size // 2
    y0 = max(0, int(y) - half)
    x0 = max(0, int(x) - half)
    y1 = min(img.shape[-2], y0 + size)
    x1 = min(img.shape[-1], x0 + size)
    if (y1 - y0) < size:
        y0 = max(0, y1 - size)
    if (x1 - x0) < size:
        x0 = max(0, x1 - size)
    return img[y0:y1, x0:x1], (y0, y1, x0, x1)


def ls_coeff(donor: np.ndarray, target: np.ndarray) -> float:
    donor_f = donor.astype(np.float32, copy=False)
    target_f = target.astype(np.float32, copy=False)
    denom = float(np.sum(donor_f * donor_f) + 1e-6)
    if denom <= 0:
        return 0.0
    num = float(np.sum(donor_f * target_f))
    return max(0.0, num / denom)


def percentile_cap(donor: np.ndarray, target: np.ndarray) -> float:
    if PERCENTILE_Q is None:
        return float('inf')
    donor_q = float(np.percentile(donor, PERCENTILE_Q))
    target_q = float(np.percentile(target, PERCENTILE_Q))
    if not np.isfinite(donor_q) or donor_q <= 1e-6:
        return float('inf')
    if not np.isfinite(target_q) or target_q < 0:
        target_q = max(0.0, target_q)
    cap = (target_q / donor_q) * float(SAFETY_FACTOR)
    if cap < 0:
        return 0.0
    return cap


def final_cap(raw_coeff: float, ratio_cap: float) -> float:
    coeff = raw_coeff
    if ratio_cap is not None:
        coeff = min(coeff, ratio_cap)
    if MAX_COEFF is not None:
        coeff = min(coeff, float(MAX_COEFF))
    return max(0.0, coeff)


def estimate_coeff_roi(target: np.ndarray, donor: np.ndarray, *, roi_cfg: Dict[str, int], pad: int) -> Tuple[float, Dict[str, Any]]:
    base_cfg = dict(roi_cfg)
    roi_target, coords = extract_roi_patch(target, **base_cfg)
    roi_donor, _ = extract_roi_patch(donor, **base_cfg)
    if pad and pad > 0:
        pad_cfg = dict(base_cfg)
        pad_cfg['size'] = base_cfg['size'] + 2 * int(pad)
        roi_target, coords = extract_roi_patch(target, **pad_cfg)
        roi_donor, _ = extract_roi_patch(donor, **pad_cfg)

    coeff_raw = ls_coeff(roi_donor, roi_target)
    cap_ratio = percentile_cap(roi_donor, roi_target) if coeff_raw > 0 else float('inf')
    coeff = final_cap(coeff_raw, cap_ratio)

    info = dict(
        coeff=float(coeff),
        coeff_raw=float(coeff_raw),
        coeff_cap_ratio=(float(cap_ratio) if np.isfinite(cap_ratio) else None),
        pad=int(pad),
        roi_coords=dict(top=int(coords[0]), bottom=int(coords[1]), left=int(coords[2]), right=int(coords[3])),
        roi_sum_raw=float(np.sum(roi_target)),
        roi_sum_donor=float(np.sum(roi_donor)),
    )
    return coeff, info


def apply_spillover(stack: np.ndarray, donor_idx: int, target_idx: int, coeff: float) -> None:
    stack[target_idx] = np.clip(stack[target_idx] - coeff * stack[donor_idx], 0, None)


def convert_to_dtype(stack: np.ndarray, dtype: np.dtype) -> np.ndarray:
    if np.issubdtype(dtype, np.integer):
        info = np.iinfo(dtype)
        out = np.clip(stack, info.min, info.max)
    else:
        out = stack
    return out.astype(dtype, copy=False)


def build_output_base(input_path: Path) -> Tuple[Path, Path, Path, Path]:
    """
    Erzeugt Output-Pfade im dedizierten spillover/-Ordner.
    
    Struktur:
    BASE_EXPORT/spillover/
        â”œâ”€â”€ fused_decon_spillover_corrected.ome.tif
        â”œâ”€â”€ spillover_coefficients.json
        â””â”€â”€ spillover_channels/
    """
    # Spillover-Ordner direkt in BASE_EXPORT (Root-bounded!)
    spillover_dir = BASE_EXPORT / "spillover"
    spillover_dir.mkdir(exist_ok=True, parents=True)
    
    # Fixer Dateiname (kein Timestamp!)
    stem = input_path.stem.replace('.ome', '')  # Entfernt .ome.tif
    ome_path = spillover_dir / f"{stem}_spillover_corrected.ome.tif"
    channel_dir = spillover_dir / "spillover_channels"
    spill_path = spillover_dir / "spillover_coefficients.json"
    
    return spillover_dir, ome_path, channel_dir, spill_path


def parse_marker_table(csv_path: Path) -> Tuple[List[Dict[str, Any]], Dict[str, int], Dict[int, Dict[str, Any]], List[Tuple[int, int]]]:
    rows: List[Dict[str, Any]] = []
    with open(csv_path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            rows.append(row)

    if not rows:
        logger.warning('Marker-CSV %s ist leer oder konnte nicht gelesen werden.', csv_path)
        return [], {}, {}, []

    headers = list(rows[0].keys())

    def _find_col(tokens: List[str], default: Optional[str] = None) -> Optional[str]:
        opts = [col for col in headers]
        if default and default in opts:
            return default
        for col in opts:
            low = col.lower()
            if any(tok in low for tok in tokens):
                return col
        return default if default in headers else None

    include_col = _find_col(['include'], None)

    include_rows: List[Dict[str, Any]] = []
    for row in rows:
        flag = row.get(include_col, 'TRUE') if include_col else 'TRUE'
        if str(flag).strip().upper() in {'TRUE', '1', 'YES', 'Y', 'JA'}:
            include_rows.append(row)

    if not include_rows:
        logger.warning('Keine Include==TRUE-EintrÃ¤ge in Marker-CSV: %s', csv_path)
        return [], {}, {}, []

    no_col = _find_col(['no'], 'No')
    marker_col = _find_col(['marker', 'antibody', 'target'], 'Marker-Name')
    fluor_col = _find_col(['fluor', 'dye'], None)
    spill_col = _find_col(['spillover'], 'Spillover_from')

    mapping: Dict[str, int] = {}
    for idx, row in enumerate(include_rows):
        channel_no = str(row.get(no_col, '')).strip() if no_col else str(idx + 1)
        if channel_no:
            mapping[channel_no] = idx

    channel_info: Dict[int, Dict[str, Any]] = {}
    spill_pairs: List[Tuple[int, int]] = []

    for idx, row in enumerate(include_rows):
        channel_no = str(row.get(no_col, '')).strip() if no_col else str(idx + 1)
        marker_name = str(row.get(marker_col, '')).strip() if marker_col else ''
        fluor_name = str(row.get(fluor_col, '')).strip() if fluor_col else ''
        spill_from = str(row.get(spill_col, '')).strip() if spill_col else ''

        channel_info[idx] = dict(
            no=channel_no or str(idx + 1),
            marker=marker_name,
            fluor=fluor_name,
            spillover_from=spill_from,
        )

        donor_no = spill_from
        if donor_no and donor_no.lower() != 'na':
            donor_idx = mapping.get(donor_no)
            if donor_idx is None:
                logger.warning('Spillover-Referenz %s (fÃ¼r Ziel No=%s) ist nicht in Include==TRUE enthalten.', donor_no, channel_no)
                continue
            spill_pairs.append((idx, donor_idx))

    return include_rows, mapping, channel_info, spill_pairs



## 3. CSV-Analyse & Index-Mapping

In [ ]:
include_rows, id_mapping, channel_info, spill_pairs = parse_marker_table(MARKER_CSV)
logger.info('Include==TRUE: %d KanÃ¤le', len(include_rows))
logger.info('Spillover-Paare (gemappt): %s', spill_pairs)

skip_zero_based = [max(0, sc - 1) for sc in SKIP_CHANNELS]
logger.info('Ãœberspringe KanÃ¤le (0-based): %s', skip_zero_based)


## 3.5 DIAGNOSE: Input-File prÃ¼fen

In [ ]:
import os

# PrÃ¼fe Input-File GrÃ¶ÃŸe
input_size_bytes = INPUT_PATH.stat().st_size
input_size_gb = input_size_bytes / (1024**3)
print(f"Input-File: {INPUT_PATH.name}")
print(f"Input-GrÃ¶ÃŸe: {input_size_gb:.2f} GB ({input_size_bytes:,} Bytes)")

# DETAILLIERTE TIFF-ANALYSE
with TiffFile(str(INPUT_PATH)) as tf:
    print(f"\nðŸ” DETAILLIERTE TIFF-ANALYSE:")
    print(f"  Anzahl Series: {len(tf.series)}")
    print(f"  Anzahl Pages: {len(tf.pages)}")
    
    # Erste Series (Standard)
    series = tf.series[0]
    print(f"\n  Series[0]:")
    print(f"    Axes: {series.axes}")
    print(f"    Shape: {series.shape}")
    print(f"    Dtype: {series.dtype}")
    
    # PrÃ¼fe, ob Pages separate Channels sind
    if len(tf.pages) > 1:
        print(f"\n  âš ï¸ WARNUNG: {len(tf.pages)} Pages gefunden!")
        print(f"     MÃ¶glicherweise sind die Channels als separate Pages gespeichert.")
        print(f"     Erste 3 Pages:")
        for i, page in enumerate(tf.pages[:3]):
            print(f"       Page {i}: Shape={page.shape}, Dtype={page.dtype}")
    
    # Berechne erwartete GrÃ¶ÃŸe WENN 83 Channels
    expected_bytes_single = 1
    for dim in series.shape:
        expected_bytes_single *= dim
    expected_bytes_single *= np.dtype(series.dtype).itemsize
    
    expected_bytes_83ch = expected_bytes_single * 83
    expected_gb_83ch = expected_bytes_83ch / (1024**3)
    
    print(f"\n  Erwartete GrÃ¶ÃŸe (1 Channel, unkomprimiert): {expected_bytes_single / (1024**3):.2f} GB")
    print(f"  Erwartete GrÃ¶ÃŸe (83 Channels, unkomprimiert): {expected_gb_83ch:.2f} GB")
    
print(f"\n{'='*60}")
print(f"ðŸš¨ DIAGNOSE:")
if len(tf.pages) > 80:
    print(f"  âœ… File hat {len(tf.pages)} Pages â†’ wahrscheinlich 83 Channels als separate Pages!")
    print(f"  âŒ ABER: tifffile.series[0] liest nur ERSTE Page/Channel!")
    print(f"  ðŸ”§ FIX: load_ome_to_chw() muss ALLE Pages laden, nicht nur series[0]!")
else:
    print(f"  âŒ File hat NUR {len(tf.pages)} Pages â†’ Script 1 hat falsch gespeichert!")
    print(f"  ðŸ”§ FIX: Script 1 muss Channels als Multi-Page TIFF oder mit C-Achse speichern!")
print(f"{'='*60}")

## 4. Spillover entfernen & speichern

In [ ]:

stack_raw, orig_dtype, channel_names = load_ome_to_chw(INPUT_PATH)
if stack_raw.shape[0] != len(include_rows):
    logger.warning('Stack-Kanalanzahl (%d) entspricht nicht Include==TRUE (%d). Bitte prÃ¼fen!', stack_raw.shape[0], len(include_rows))

stack_work = stack_raw.copy()

_unknown_tokens = {'nan', 'none', 'unknown', ''}

def _clean_text(value: Any) -> str:
    if value is None:
        return ''
    text = str(value).strip()
    if not text or text.lower() in _unknown_tokens:
        return ''
    return text

def _channel_label(idx: int) -> str:
    info = channel_info.get(idx, {})
    marker = _clean_text(info.get('marker'))
    fluor = _clean_text(info.get('fluor'))
    parts = [p for p in (marker, fluor) if p]
    if parts:
        return ' | '.join(parts)
    if idx < len(channel_names):
        return channel_names[idx]
    return f'C{idx+1:02d}'

def _safe_token(text: str) -> str:
    cleaned = text.replace('|', '_').replace('/', '_').replace('\\', '_')
    cleaned = cleaned.replace(' ', '_')
    return ''.join(ch if ch.isalnum() or ch in {'_', '-'} else '_' for ch in cleaned)

channel_labels = [_channel_label(i) for i in range(stack_work.shape[0])]

logger.info('Starte Spillover-Korrektur mit ROI-LS (V7-Mechanik).')
spill_results: List[Dict[str, Any]] = []
for target_idx, donor_idx in tqdm(spill_pairs, desc='Spillover'):
    if target_idx >= stack_work.shape[0] or donor_idx >= stack_work.shape[0]:
        logger.warning('Paar auÃŸerhalb des Stacks: target=%d donor=%d', target_idx, donor_idx)
        continue
    if target_idx in skip_zero_based or donor_idx in skip_zero_based:
        logger.info('Ãœberspringe Paar target=%d donor=%d (Skip-Liste)', target_idx, donor_idx)
        continue

    roi_before, _ = extract_roi_patch(stack_work[target_idx], **ROI_CFG)
    coeff, stats = estimate_coeff_roi(
        stack_work[target_idx],
        stack_work[donor_idx],
        roi_cfg=ROI_CFG,
        pad=ROI_PAD,
    )
    apply_spillover(stack_work, donor_idx, target_idx, coeff)
    roi_after, _ = extract_roi_patch(stack_work[target_idx], **ROI_CFG)

    target_info = channel_info.get(target_idx, {})
    donor_info = channel_info.get(donor_idx, {})
    target_marker = _clean_text(target_info.get('marker')) or (channel_names[target_idx] if target_idx < len(channel_names) else f'C{target_idx+1:02d}')
    donor_marker = _clean_text(donor_info.get('marker')) or (channel_names[donor_idx] if donor_idx < len(channel_names) else f'C{donor_idx+1:02d}')
    target_fluor = _clean_text(target_info.get('fluor'))
    donor_fluor = _clean_text(donor_info.get('fluor'))

    stats.update(
        target_idx=int(target_idx),
        donor_idx=int(donor_idx),
        roi_sum_after=float(np.sum(roi_after)),
        roi_mean_before=float(np.mean(roi_before)),
        roi_mean_after=float(np.mean(roi_after)),
        target_marker=target_marker,
        target_fluor=target_fluor,
        donor_marker=donor_marker,
        donor_fluor=donor_fluor,
    )
    spill_results.append(stats)

    ratio_cap_disp = stats.get('coeff_cap_ratio')
    if ratio_cap_disp is None or ratio_cap_disp < 0:
        ratio_cap_disp = float('inf')
    target_desc = target_marker if not target_fluor else f"{target_marker} [{target_fluor}]"
    donor_desc = donor_marker if not donor_fluor else f"{donor_marker} [{donor_fluor}]"
    logger.info(
        'C%d (%s) <- C%d (%s) | coeff=%.4f (raw %.4f, cap %.4f) | ROI-mean %.3fâ†’%.3f',
        target_idx + 1,
        target_desc,
        donor_idx + 1,
        donor_desc,
        stats['coeff'],
        stats['coeff_raw'],
        ratio_cap_disp,
        stats['roi_mean_before'],
        stats['roi_mean_after'],
    )

logger.info('Spillover-Korrektur abgeschlossen. Konvertiere zurÃ¼ck in %s.', orig_dtype)

data_out = convert_to_dtype(stack_work, orig_dtype)
spillover_dir, ome_path, channel_dir, spill_path = build_output_base(INPUT_PATH)
channel_dir.mkdir(parents=True, exist_ok=True)

metadata_labels = channel_labels if channel_labels else channel_names
metadata = {'axes': 'CYX', 'Channel': {'Name': metadata_labels}}
tiff.imwrite(str(ome_path), data_out, photometric='minisblack', metadata=metadata)
logger.info('OME-TIFF gespeichert: %s', ome_path)

for ci in range(data_out.shape[0]):
    info = channel_info.get(ci, {})
    marker_name = _clean_text(info.get('marker')) or (channel_names[ci] if ci < len(channel_names) else f'C{ci+1:02d}')
    fluor_name = _clean_text(info.get('fluor'))
    parts = [p for p in (marker_name, fluor_name) if p]
    if not parts:
        parts = [channel_labels[ci]]
    label_core = '_'.join(_safe_token(part) for part in parts)
    label = f"C{ci+1:03d}_{label_core}" if label_core else f"C{ci+1:03d}"
    tiff.imwrite(str(channel_dir / f"{label}.tif"), data_out[ci], photometric='minisblack')
logger.info('KanÃ¤le exportiert: %s', channel_dir)

with open(spill_path, 'w', encoding='utf-8') as f:
    json.dump(
        dict(
            pairs=spill_results,
            params=dict(
                roi=ROI_CFG,
                pad=ROI_PAD,
                percentile_q=PERCENTILE_Q,
                safety_factor=SAFETY_FACTOR,
                max_coeff=MAX_COEFF,
            ),
        ),
        f,
        indent=2,
    )
logger.info('Spillover-Log gespeichert: %s', spill_path)

logger.info('Workflow fertig.')
logger.info('  OME  : %s', ome_path)
logger.info('  ChDir: %s', channel_dir)
logger.info('  Spill: %s', spill_path)



## 5. Ergebnis-Check

In [ ]:
print('OME-TIFF:', ome_path)
print('Channel-Verzeichnis:', channel_dir)
print('Spillover-Log:', spill_path)
